# Bokeh
Python Bokeh - это библиотека визуализации данных, которая позволяет создавать интерактивные диаграммы и графики. Она отображает их с использованием HTML и JavaScript, поэтому ее часто используют для создания интерактивных визуализаций для современных веб-браузеров. Однако это еще и не менее мощный инструмент для изучения и понимания данных из датасетов или создания красивых пользовательских диаграмм для проекта или отчета.

Для того чтобы привести наиболее наглядные и прикладные примеры использования библиотеки, я буду строить графики не для рандомных чисел, а для определенного датасета, в котором содержатся данные о биллионерах на 2023-й год.

Как уже было сказано, Bokeh чаще всего используется для построения графиков в веб-браузере (для этого достаточно написать в коде `output_file('filename.html)' ` - и тогда график сохранится в html-формате). Однако цель моего докалада - описать возможности, предоставляемые библиотекой, поэтому не вижу смысла для каждого кусочка кода создавать html-файл и запускать его в браузере - будем отображать графики прямо в ноутбуке (за это отвечает строчка `output_notebook()`)

Данные взяты с kaggle: https://www.kaggle.com/code/alancano/billionaires-statistics-2023/input

In [2]:
import pandas as pd # импортируем необходимые библиотеки
import numpy as np
import math
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import row
from dateutil.parser import parse
from bokeh.tile_providers import get_provider, Vendors
from pyproj import Proj, transform

output_notebook() # отображаем графики в ноутбуке

data = pd.read_csv('https://raw.githubusercontent.com/katrinnx/Deep_Python_2023/main/Billionaires%20Statistics%20Dataset.csv')
data.head()

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
0,1,211000,Fashion & Retail,Bernard Arnault & family,74.0,France,Paris,LVMH,Fashion & Retail,France,...,1.1,"$2,715,518,274,227",65.6,102.5,82.5,24.2,60.7,67059887.0,46.227638,2.213749
1,2,180000,Automotive,Elon Musk,51.0,United States,Austin,"Tesla, SpaceX",Automotive,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
2,3,114000,Technology,Jeff Bezos,59.0,United States,Medina,Amazon,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
3,4,107000,Technology,Larry Ellison,78.0,United States,Lanai,Oracle,Technology,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,328239523.0,37.090240,-95.712891


> **Линейный график**

Для начала построим линейный график по точкам: для этого используется функция `f.line(.)`

Посмотрим на среднее состояние людей по возрасту:

In [ ]:
data_age = data.groupby('age').agg({'finalWorth': 'mean'}).reset_index()
data_age.head()

,age,finalWorth
0,18.0,3500.0
1,19.0,1700.0
2,20.0,2300.0
3,21.0,2600.0
4,26.0,1450.0


In [ ]:
f = figure(title='Distribution of worth by age',
           x_axis_label='Age',
           y_axis_label='Average worth') # описание фигуры: название графика и осей

f.line(x=data_age.age, y=data_age.finalWorth, # рисуем линию
       line_width = 2,
       color = "#065C27",
       alpha=0.8)

show(f) # отображаем график

Заметим, что при пострении графиков с использованием Bokeh справа будут расположены кнопки, добавляющие интерактивность (и их не надо никак описывать в коде - они по умолчанию есть в библиотеке).

При нажатии на соответствующий значок будет активирована возможность двигать график так, как позволяет кнопка (причем может быть выбрано несколько опций сразу).

Опишем функционал кнопок по порядку (сверху вниз):


1.   Перемещение графика
2.   Выбор и зум какой-то области графика
3.   Приближение/отдаление
4.   Сохранение в .png формате
5.   Возвращение к изначальному масштабу и расположению
6.   Документация



График можно сделать более наглядным: точек, по которым мы строим линию, не так много, поэтому отметим их кружочками. Кроме того, создадим легенду и благодаря функции `f.legend.click_policy='hide'` сделаем так, чтобы при нажатии на названия в легенде соответствующие части графика исчезали или появлялись. То есть мы получим и lineplot, и scatterplot на одном графике

In [ ]:
f = figure(title='Distribution of worth by age',
           x_axis_label='Age',
           y_axis_label='Average worth')

f.line(x=data_age.age, y=data_age.finalWorth,
       line_width = 2,
       color = "#065C27",
       alpha = 0.8,
       legend_label = 'line')

f.circle(data_age.age, data_age.finalWorth,
         size=6,
         color='#C62E65',
         alpha = 0.8, legend_label='circle') # накладываем точки

f.legend.location = 'top_left' # расположим легенду слева сверху

f.legend.click_policy='hide' # добавляем возможность выбирать, какой график отображать (делаем легенду интерактивной)

show(f)

Теперь сделаем так, чтобы при наведении на точку отображался возраст и среднее состояние миллиардера - добавим еще больше интерактивности:

In [ ]:
source = ColumnDataSource(data=data_age) # создаем ColumnDataSource из нашего датасета data_age

f = figure(title='Distribution of worth by age',
           x_axis_label='Age',
           y_axis_label='Average worth')

f.line(source=source, x='age', y='finalWorth',
       line_width=2, color="#065C27", alpha=0.8, legend_label='line')

f.circle(source=source, x='age', y='finalWorth',
         size=6, color='#C62E65', alpha=0.8, legend_label='circle')

f.legend.location = 'top_left'
f.legend.click_policy = 'hide'

hover = HoverTool()
hover.tooltips = [("Age", '@age'), ("Average worth", '@finalWorth')] # добавляем интерактивную информацию о возрасте и состоянии

f.add_tools(hover)

show(f)

Интересно, что самая верхняя точка (то есть самое большое состояние) соответствует возрасту 92. Наверное, только один человек в этом возрасте является миллиардером, и его состояние сильно больше, чем у остальных. Проверим:

In [3]:
data[data.age == 92]

,rank,finalWorth,category,personName,age,country,city,source,industries,countryOfCitizenship,...,cpi_change_country,gdp_country,gross_tertiary_education_enrollment,gross_primary_education_enrollment_country,life_expectancy_country,tax_revenue_country_country,total_tax_rate_country,population_country,latitude_country,longitude_country
4,5,106000,Finance & Investments,Warren Buffett,92.0,United States,Omaha,Berkshire Hathaway,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
98,99,17100,Media & Entertainment,Rupert Murdoch & family,92.0,United States,New York,"Newspapers, TV network",Media & Entertainment,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
377,365,6700,Finance & Investments,George Soros,92.0,United States,Katonah,Hedge funds,Finance & Investments,United States,...,7.5,"$21,427,700,000,000",88.2,101.8,78.5,9.6,36.6,3.282395e+08,37.090240,-95.712891
432,425,6000,Finance & Investments,Frank Lowy,92.0,Israel,Tel Aviv,Investments,Finance & Investments,Australia,...,0.8,"$395,098,666,122",63.4,104.9,82.8,23.1,25.3,9.053300e+06,31.046051,34.851612
1034,1027,2900,Sports,Bernard Ecclestone & family,92.0,United Kingdom,London,Formula One,Sports,United Kingdom,...,1.7,"$2,827,113,184,696",60.0,101.2,81.3,25.5,30.6,6.683440e+07,55.378051,-3.435973
1128,1104,2700,Diversified,Mustafa Rahmi Koc,92.0,Turkey,Istanbul,Diversified,Diversified,Turkey,...,15.2,"$754,411,708,203",23.9,93.2,77.4,17.9,42.3,8.342962e+07,38.963745,35.243322
1250,1217,2500,Automotive,Lachhman Das Mittal,92.0,India,Delhi,Tractors,Automotive,India,...,7.7,"$2,611,000,000,000",28.1,113.0,69.4,11.2,49.7,1.366418e+09,20.593684,78.962880


Гипотеза не подтвердилось - в мире целых семь 92-летних миллиардеров. Однако, как мы и предполагали, очень богатый только один из них - Уоррен Баффетт, американский предприниматель (и за счет него сильно подскочило среднее состояние для значения age = 92).

Кстати, на момент создания этого доклада ему уже 93 года)

> **Scatter plot**

Точечный график может быть представлен не только в виде кружочков, но еще и треугольников или квадратов.

Однако для демонстрации этой возможности неудобно будет рисовать похожие графики отдельно (один под другим): для решения этой проблемы в Bokeh есть еще одна полезная возможность - располагать графики рядом друг с другом в строчку(то есть создавать subplots); для этого используется метод `row()`

In [ ]:
source = ColumnDataSource(data=data_age)

f1 = figure(width=500, height=400, # сначала описываем график 1 (с треугольниками)
            title='Distribution of worth by age',
           x_axis_label='Age',
           y_axis_label='Average worth')

f1.triangle(source=source, x='age', y='finalWorth',
         size=6, color='#2F1847', legend_label='triangle')


f2 = figure(width=500, height=400, # теперь описываем график 2 (с квадратиками)
            title='Distribution of worth by age',
           x_axis_label='Age',
           y_axis_label='Average worth')

f2.square(source=source, x='age', y='finalWorth',
         size=6, color='#624763', legend_label='square')


hover = HoverTool()
hover.tooltips = [("Billionaires - ", "Age: @age / Average worth: @finalWorth")]
hover.mode = 'vline'

f1.add_tools(hover)
f2.add_tools(hover)

p = row(f1, f2) # выставляем графики в строчку

show(p) # выводим результат

Видно, что для наших данных такой scatterplot намного менее нагляден по сравнению с тем, в котором мы рисуем еще и линейную зависимость между точками

>**Гистограмма**

>Для начала рассмотрим горизонтальную гистограмму

Данный тип графика подойдет для изучения распределения, когда рассматривается много категорий: по оси OY ставим категории (по осb OX было бы наложение), а по оси OX - числовые значения.

Проанализируем распределение миллиардеров по сферам, в которых они зарабатывают деньги:

In [ ]:
data_category = data.groupby('category').agg({'personName': 'nunique'}).reset_index()
data_category = data_category.sort_values(by=['personName'])

In [ ]:
p = figure(title='Distribution of billionaires by categories',
           y_range=data_category.category,
           x_axis_label='Number of billionaires',
           y_axis_label='Category of billionaire`s business',
           width=800, height=600)

p.hbar(y=data_category.category, right=data_category.personName, #создаем horizontal bar
       width=0.5, color='blue', alpha=0.5,
       height=0.4) # параметр height - отступ между значениями по оси OY (остальные параметры уже известны)

show(p)

Получается, большинство миллиардеров сколотили свое состояние в сфере финансов и инвестиций

>Теперь перейдем к вертикальной гистограмме

Данный тип графика подойдет для изучения распределения, когда рассматривается мало категорий: по оси OX ставим категории, а по оси OY - числовые значения.

Рассмотрим распределение self-made и не self-made миллиардеров:

In [21]:
data_slfmade = data.groupby('selfMade').agg({'personName': 'nunique'}).reset_index()

In [20]:
p = figure(title='Distribution of billionaires by gender',
           x_range=data_slfmade.selfMade.astype(str),
           x_axis_label='Self-made',
           y_axis_label='Number of billionaires',
           width=800, height=600)

p.vbar(x=data_slfmade.selfMade.astype(str), top=data_slfmade.personName, #создаем vertical bar
       width=0.7, color='#6FD08C') # width - ширина столбцов

show(p)

>**Pie chart**

Рассмотрим еще один тип графиков, поддерживаемый библиотекой Bokeh - pie chart

Для начала создадим простенький график - распределение количества миллиардеров по полу

In [41]:
data_gender = data.groupby('gender').agg({'personName': 'nunique'}).reset_index()
total_num = data_gender.personName.sum()
data_gender['perc'] = data_gender.personName * 100 / total_num
data_gender

,gender,personName,perc
0,F,304,11.8611
1,M,2259,88.1389


In [42]:
def renam(s): # переименуем значения в столбце gender, чтобы на графике легенда была нагляднее
  if s == "F":
    s = "Female"
  else:
    s = "Male"
  return s
data_gender.gender = data_gender.gender.apply(lambda x: renam(x))
data_gender

,gender,personName,perc
0,Female,304,11.8611
1,Male,2259,88.1389


In [43]:
graph = figure(title = "Distribution of billionaires by gender")

sectors = data_gender.gender # задаем имена секторов

percentages = data_gender.perc # задаем веса секторов

radians = [math.radians((percent / 100) * 360) for percent in percentages] # переводим в радианы

start_angle = [math.radians(0)] # вычисляем углы для построения pie chart
prev = start_angle[0]
for i in radians[:-1]:
    start_angle.append(i + prev)
    prev = i + prev
end_angle = start_angle[1:] + [math.radians(0)]

x = 0 # центр графика
y = 0

radius = 0.8 # задаем радиус

color = ["#F79DDF", "#32CBFF"] # цвета секторов

for i in range(len(sectors)): # строим pie chart
    graph.wedge(x, y, radius,
                start_angle = start_angle[i],
                end_angle = end_angle[i],
                color = color[i],
                legend_label = str(sectors[i]))

show(graph)

Pie chart тоже можно сделать интерактивным, однако в этом нет особой необходимости, если количество секторов небольшое (см. прошлый пример - график и так нагляден).

Поэтому теперь построим график с распределением количества миллиардеров по дню, в который они родились

In [27]:
data = data.dropna(subset=['birthDate'])
data['day_of_birth'] = data.birthDate.apply(lambda x: parse(str(x)).day)

In [28]:
data_days = data.groupby('day_of_birth').agg({'personName': 'nunique'}).reset_index()
data_days.head()

,day_of_birth,personName
0,1,695
1,2,69
2,3,68
3,4,57
4,5,62


In [29]:
total = data_days.personName.sum()
data_days["perc"] = data_days.personName * 100 / total

In [40]:
graph = figure(title="Distribution of billionaires by the day of birth", width=800, height=800)

sectors = data_days.day_of_birth

percentages = data_days.perc

radians = [math.radians((percent / 100) * 360) for percent in percentages]

start_angle = [math.radians(0)]
prev = start_angle[0]
for i in radians[:-1]:
    start_angle.append(i + prev)
    prev = i + prev
end_angle = start_angle[1:] + [math.radians(0)]

x = 0
y = 0

radius = 0.8

# задаем цветовую палитру
colors = ["#52b788", "#c526ff", "#9b8885", "#25127a", "#fdd692", "#cda2db", "#d95d39", "#718e22"," #f2b5e7", "#a7fc8d",
          "#d37ac3", "#3a0ab2", "#8e8229", "#437a6f", "#e2e58b", "#5c2660", "#0f4c5c", "#3d3737", "#31ce5b", "#2b092b",
          "#a17ca8",  "#4a824b", "#935d66", "#8f50fc", "#a04668", "#8faa86", "#ba1509", "#3a3436", "#6f774c", "#3e754c",
          "#dd635f"
         ]

source = ColumnDataSource(data=dict( # cоздаем источник данных ColumnDataSource, чтобы можно было добавить интерактивность
    start_angle=start_angle,
    end_angle=end_angle,
    color=colors,
    sectors=sectors,
    percentages=percentages
))

sources = []
for i in range(len(sectors)): # описываем каждый сектор
    source = ColumnDataSource(data=dict(
        start_angle=[start_angle[i]],
        end_angle=[end_angle[i]],
        color=[colors[i]],
        sectors=[sectors[i]],
        percentages=[percentages[i]]
    ))
    sources.append(source)

for i in range(len(sectors)): # рисуем каждый сектор
    graph.wedge(x=x, y=y, radius=radius,
                start_angle='start_angle',
                end_angle='end_angle',
                color='color',
                legend_label=str(sectors[i]),
                source=sources[i])

hover = HoverTool() # добавляем инструмент HoverTool
tooltips = [("Day", '@sectors'), ("Percentage", '@percentages{0.2f}%')]
hover.tooltips = tooltips
graph.add_tools(hover)

show(graph)

Сам по себе график не очень нагляден, поскольку у нас слишком много секторов, однако благодаря инерактивности он выглядит намного лучше - при наведении на соответствующий кусочек pie chart видим возраст и процент миллиардеров, родившихся в этот день. Кроме того, я постаралсь подобрать цветовую палитру так, чтобы секторы были четко отделены друг от друга и не было похожих цветов.

Получили довольно интересный результат - чуть больше четверти миллиардеров со всего мира родились первого числа

> **Patch plot**

Рассмотрим новый тип графиков -  patch plot. Он используется для визуализации замкнутых многоугольных областей, часто применяется в картографии и географии для представления географических регионов или формирования визуальных элементов на основе координатных данных

В нашем датасете есть данные о координатах стран миллиардеров. Используем их для построения patch-plota: отобразим схематичное распределение миллиардеров по миру

In [ ]:
graph = figure(title = "Location of billionaires around the world", x_axis_label = "Longitude", y_axis_label = "Latitude")

df = data[["longitude_country", "latitude_country", "personName", "country"]] # выделяем нужные столбцы
df = df.groupby(['longitude_country', 'latitude_country', 'country']).agg({'personName': 'nunique'}).reset_index()

x = df.longitude_country
y = df.latitude_country

graph.patch(x, y, color='#00BD9D') # строим patch plot

show(graph)

> **Maps**

Библиотека также поддерживает работу с картами.

Поскольку, как уже было сказано, у нас есть информация о координатах стран миллиардеров, отобразим их точечно на карте

In [ ]:
inProj = Proj(init='epsg:3857') # для корректного отображения координат нужно будет конвертировать широту и долготу в исходную картографическую проекцию координат x и y
outProj = Proj(init='epsg:4326') # для этого воспользуемся бибилотекой Proj

lons, lats = [], []

for lon, lat in list(zip(df["longitude_country"], df["latitude_country"])):
    x, y = transform(outProj, inProj, lon, lat) # конвертируем
    lons.append(x)
    lats.append(y)

df["MercatorX"] = lons # сохраняем координаты
df["MercatorY"] = lats

# Renaming Columns for Tooltips
df = df.rename(columns={"personName": "number_of_billionaires"}) # для удобства переименуем столбец, в котором храним количество биллионеров в каждой стране

tile_provider = get_provider(Vendors.CARTODBPOSITRON) # загружаем карту

m = figure(title='Billionaires in different countries', width=650, # создаем карту
           height=400, x_range=(-12000000, 9000000),
           y_range=(-1000000, 7000000),
           x_axis_type='mercator', y_axis_type='mercator') # используем проекцию Меркатора

m.add_tile(tile_provider)
m.circle(x='MercatorX', y='MercatorY', size=5, color='#231942', alpha=0.8, source=df) # добавляем точки

show(m)

/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/local/lib/python3.10/dist-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
<ipython-input-141-e3b75471e7b5>:7: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = transform(outProj, inProj, lon, lat) # конв

Если соединить точки на карте (рассматриваем все континенты), то получится именно та фигура, которая была пострена с помощью patch plot

Карту нарисовали. Однако без зума неясно, какая именно страна отмечена точкой, а также не видно, сколько миллиардеров находится в отмеченной стране. Сделаем точки интерактивными

In [ ]:
# предшествующие шаги нет смысла заново описывать - они уже были проделаны в прошлой ячейке кода

tooltips = [("Number of billionaires", "@number_of_billionaires"), ("Country", "@country")] # добавляем tooltips

m = figure(title='Billionaires in different countries', width=650, # строим новый график
           height=400, x_range=(-12000000, 9000000),
           y_range=(-1000000, 7000000),
           x_axis_type='mercator', y_axis_type='mercator',
           tooltips=tooltips)

m.add_tile(tile_provider)
m.circle(x='MercatorX', y='MercatorY', size=5, color='#231942', alpha=0.8, source=df)

show(m)

Теперь при наведении на точку мы видим название страны и количество миллиардеров в ней.

Самое большое количество находится в USA - 754 человека

>**Widgets and DOM elements**

В библиотеке есть еще много прикольных элементов, но они применимы скорее уже не для анализа данных, а для оформления веб-страниц. Однако я все равно приведу несколько примеров, которые будут корректно работать и в ноутбуке, но уже не буду обращаться к нашему датасету, а продемонстрирую возможности библиотеки для рандомных чисел.

> С помощью `ColorPicker` пользователь может сам задавать цвет для графика как с помощью RGB, так и просто с помощью палитры

In [22]:
from bokeh.layouts import column
from bokeh.models import ColorPicker

plot = figure(x_range=(0, 1), y_range=(0, 1), width=350, height=350)
line = plot.line(x=(0,1), y=(0,1), color="black", line_width=4)

picker = ColorPicker(title="Line Color")
picker.js_link('color', line.glyph, 'line_color')

show(column(plot, picker))

>Bokeh также поддерживает использование вкладок, на которых можно отображать разные графики

In [23]:
from bokeh.models import CustomJS, TabPanel, Tabs

p1 = figure(width=300, height=300) # создаем первый график
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="#138A36", alpha=0.5)
tab1 = TabPanel(child=p1, title="circle") # первая вкладка

p2 = figure(width=300, height=300) # создаем второй график
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="#138A36")
tab2 = TabPanel(child=p2, title="line") # вторая вкладка

show(Tabs(tabs=[tab1, tab2])) # соединяем

>С помощью Bokeh можно создать поле для ввода пароля

In [24]:
from bokeh.models import PasswordInput

password_input = PasswordInput(placeholder="enter password...")
password_input.js_on_change("value", CustomJS(code="""
    console.log('password_input: value=' + this.value, this.toString())
"""))

show(password_input)

Однако для того, чтобы использовать такие штуки на сайтах, только Bokeh, конечно, недостаточно. Нужно будет писать html-код для внедрения этих виджетов в веб-страницы

>Еще можно предоставить пользователю возможность выбирать размер точек на scatterplot

In [ ]:
from bokeh.models import Spinner

x = np.random.rand(10)
y = np.random.rand(10)

p = figure(x_range=(0, 1), y_range=(0, 1))
points = p.scatter(x=x, y=y, size=4, color='#3C1742',alpha=0.8)

spinner = Spinner(title="Glyph size", low=1, high=40, step=0.5, value=4, width=80)
spinner.js_link('value', points.glyph, 'size')

show(row(column(spinner, width=100), p))

>А еще можно добавить переключатель

In [ ]:
from bokeh.models import Switch

switch = Switch(active=True)
switch.js_on_change("active", CustomJS(code="""
    console.log('switch: active=' + this.active, this.toString())
"""))
show(switch)